In [16]:
from google.colab import files
uploaded = files.upload()


Saving dataset_cat_dog.zip to dataset_cat_dog.zip


In [17]:
import zipfile
import io

zip_ref = zipfile.ZipFile(io.BytesIO(uploaded['dataset_cat_dog.zip']), 'r')
zip_ref.extractall()
zip_ref.close()


In [32]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [33]:
# Path dataset
dataset_path = "./dataset_cat_dog"
train_path = "./dataset_cat_dog/train"
test_path = "./dataset_cat_dog/test"

In [34]:
# Dimensi gambar
img_width = 150
img_height = 150

# Load and preprocess images
def load_images_from_folder(folder, label):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (img_width, img_height))
            img = img.astype('float32') / 255.0
            images.append([img, label])
    return images

In [35]:
# Load images from cats folder
cats_folder = os.path.join(train_path, "cats")
cat_images = load_images_from_folder(cats_folder, 0)  # Label 0 for cats


In [36]:
# Load images from dogs folder
dogs_folder = os.path.join(train_path, "dogs")
dog_images = load_images_from_folder(dogs_folder, 1)  # Label 1 for dogs

In [37]:
# Combine cat and dog images
images = cat_images + dog_images
random.shuffle(images)

In [38]:
# Split images and labels
data = np.array([img_label[0] for img_label in images], dtype=np.float32)
labels = np.array([img_label[1] for img_label in images], dtype=np.float32)

In [39]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


In [40]:
# CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [41]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
# Train the model
batch_size = 32
epochs = 10

model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/10
17/17 [==============================] - 31s 2s/step - loss: 0.7130 - accuracy: 0.5944 - val_loss: 0.6676 - val_accuracy: 0.6194
Epoch 2/10
17/17 [==============================] - 29s 2s/step - loss: 0.6471 - accuracy: 0.6374 - val_loss: 0.6409 - val_accuracy: 0.6343
Epoch 3/10
17/17 [==============================] - 30s 2s/step - loss: 0.5924 - accuracy: 0.7028 - val_loss: 0.7141 - val_accuracy: 0.6119
Epoch 4/10
17/17 [==============================] - 30s 2s/step - loss: 0.5897 - accuracy: 0.6879 - val_loss: 0.6193 - val_accuracy: 0.6418
Epoch 5/10
17/17 [==============================] - 30s 2s/step - loss: 0.5017 - accuracy: 0.7664 - val_loss: 0.6763 - val_accuracy: 0.6493
Epoch 6/10
17/17 [==============================] - 30s 2s/step - loss: 0.4405 - accuracy: 0.8150 - val_loss: 0.7030 - val_accuracy: 0.6642
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.3952 - accuracy: 0.8206 - val_loss: 0.7078 - val_accuracy: 0.6642
Epoch 8/10
17/17 [==

In [43]:
# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 2s 335ms/step - loss: 0.9626 - accuracy: 0.6866
Loss:  0.9626293182373047
Accuracy:  0.6865671873092651


In [44]:
from sklearn.metrics import classification_report

# Melakukan prediksi pada set pengujian
predictions = model.predict(test_data)
predictions = np.round(predictions).flatten()

# Menghasilkan laporan klasifikasi
report = classification_report(test_labels, predictions, target_names=['cats', 'dogs'])
print(report)


5/5 [==============================] - 4s 525ms/step
              precision    recall  f1-score   support

        cats       0.84      0.61      0.71        83
        dogs       0.56      0.80      0.66        51

    accuracy                           0.69       134
   macro avg       0.70      0.71      0.68       134
weighted avg       0.73      0.69      0.69       134

